# Control Plots

Author(s): Raghav Kansal

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh import utils, hh_vars, plotting
from boostedhh.utils import PAD_VAL
from bbtautau.postprocessing import Samples, postprocessing
from bbtautau.postprocessing.Samples import CHANNELS, SAMPLES, SIGNALS
import bbtautau.postprocessing.utils as putils

import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("boostedhh.utils")
logger.setLevel(logging.DEBUG)

In [2]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [3]:
MAIN_DIR = Path("../../../")
CHANNEL = CHANNELS["hh"]  # options: "hh", "he", "hm"

plot_dir = MAIN_DIR / f"plots/ControlPlots/25Apr17{CHANNEL.key}"
plot_dir.mkdir(parents=True, exist_ok=True)

year = "2022"

base_dir = Path("/ceph/cms/store/user/rkansal/bbtautau/skimmer/")
data_paths = {
    "signal": base_dir / "25Apr17bbpresel_v12_private_signal",
    "data": base_dir / "25Apr17bbpresel_v12_private_signal",
    "bg": base_dir / "25Apr17bbpresel_v12_private_signal",
}

sigs = {s + CHANNEL.key: SAMPLES[s + CHANNEL.key] for s in SIGNALS}
bgs = {bkey: b for bkey, b in SAMPLES.items() if b.get_type() == "bg"}

## Load samples

In [ ]:
# dictionary that will contain all information (from all samples)
events_dict = postprocessing.load_samples(
    year,
    CHANNEL,
    data_paths,
    load_data=True,
    load_bgs=True,  # , filters_dict=postprocessing.bb_filters(num_fatjets=3)
)
cutflow = pd.DataFrame(index=list(events_dict.keys()))

utils.add_to_cutflow(events_dict, "Preselection", "finalWeight", cutflow)
cutflow

Triggers

In [ ]:
postprocessing.apply_triggers(events_dict, year, CHANNEL)
utils.add_to_cutflow(events_dict, "Triggers", "finalWeight", cutflow)
cutflow

bbtautau assignment

In [ ]:
bbtt_masks = postprocessing.bbtautau_assignment(events_dict, CHANNEL)

## Control Plots

In [ ]:
putils.get_var(events_dict["bbtthh"], bbtt_masks["bbtthh"], "ttFatJetPt")

In [ ]:
cutlabel = (
    r"$\geq 1$ AK8 jet with $p_T > 250$ & $m_{reg} > 50$ GeV"
    "\n"
    r"$\geq 1$ AK8 jet with $T_{Xbb} > 0.3$"
    "\n"
    r"$\geq 2$ AK8 jets with $p_T > 200$ GeV"
)

postprocessing.control_plots(
    events_dict,
    CHANNEL,
    sigs,
    bgs,
    postprocessing.control_plot_vars,
    plot_dir,
    year,
    bbtt_masks=bbtt_masks,
    cutlabel=cutlabel,
    plot_significance=True,
    show=True,
)